In [1]:
import json
import re
import os
import glob
import numpy as np
import pandas as pd

import sys
sys.path.append('..')

from calcimetry.mongo_api import MongoInfo, MongoAPI

In [83]:
mongo_info = MongoInfo()

dirhead = '/work/armitagj/data/csvs/REP4/'
drillfiles = [ f.path for f in os.scandir(dirhead) if f.is_dir() ]
filename = 'imgs2.csv'
mesuname = 'mesu.csv'

# location of calcimetry files fo case where mesu.csv does not exist
calcihead = '/work/armitagj/data/Excell_and_Photos/'

checkfile = []
checkexcel = []

# loop through all directories on the local system
for drill in drillfiles:

    # grab the only diretory (not always called "Photos")
    images = [ f.path for f in os.scandir(drill) if f.is_dir() ]
    csvfile = images[0] + '/' + filename
    calcifile = images[0] + '/' + mesuname

    try:
        # If the images have been processed
        if os.path.isfile(csvfile):

            # get the database DrillName from the directory name (some
            # DrillName entries have extra numbers)
            with MongoAPI(mongo_info=mongo_info) as mongo_api:
                drillname = drill.split('/')[-1]
                partialdrillname = f'.*{drillname}.*'
                doc = mongo_api.db['images'].find_one({'DrillName': {'$regex' : partialdrillname}})
                realdrillname = doc['DrillName']

                if mongo_api.db['measurements'].find_one({'DrillName': realdrillname}) is None:
                    # If Renaud has already created a mesu.csv file
                    if os.path.isfile(calcifile):

                        # read file
                        df = pd.read_csv(calcifile, delimiter=';')
                        
                        if len(df) < 1:
                            checkfile.append(drillname)
                            print(f'{drillname} : mesu.csv has {len(df)} rows')

                        del df

                    # Otherwise let's create the mesu.csv file from the Excel one
                    # NOTE: I don't have the mesurement ID for now, fix this
                    # once all the data is in the database
                    
                    elif drillname == 'SUG1201' or drillname == 'SUG1202':
                        df1 = pd.read_csv(excell2csved[0], sep=',')
                        df2 = pd.read_csv(csvfile, sep=';', encoding='cp1252')
                    
                    
                    else:
                        print('creating')

                        # grab the csv file I create with exell2csv.py
                        excell2csved = glob.glob(calcihead +
                                                 drill.split('/')[-1] +
                                                 '/*.csv')

                        print(excell2csved)
                        df1 = pd.read_csv(excell2csved[0], sep=',')
                        df2 = pd.read_csv(csvfile, sep=',')

                        # Modify Renaud's code to create a local mesu.csv
                        # file, so that it is identical to those already
                        # created

                        # I don't have a unique ID for the measurement,
                        # fix this once the database is complete
                        id = np.nan

                        with open(calcifile, 'w') as g:
                            g.write("MeasureId;ImageId;CalciCote;CalciVals1m;CalciVals15m\n")
                            for i, row2 in df2.iterrows():
                                imgId = row2["ImageId"]
                                cote0 = row2["Cote0"]
                                cote1 = row2["Cote1"]
                                for j, row1 in df1.iterrows():
                                    c = np.rint(row1["Cote"] * 100)
                                    if cote0 <= c and c <= cote1:
                                        g.write(f'{id};{imgId};{c};'
                                                f'{df1.iloc[j, 2]};'
                                                f'{df1.iloc[j, 4]}\n')
                            g.close()

                        # read file back in (I know...)
                        df = pd.read_csv(calcifile, delimiter=';')

                        print(f'{drillname} : new mesu.csv has {len(df)} rows')
                        
                        if len(df) < 1:
                            checkexcel.append(drillname)

                        del df1, df2, df
                else:
                    # If Renaud has already created a mesu.csv file
                    if os.path.isfile(calcifile):

                        # read file
                        df = pd.read_csv(calcifile, delimiter=';')
                        
                        
                        if len(df) < 1:
                            checkfile.append(drillname)
                            print(f'{drillname} : mesu.csv has {len(df)} rows')

                        del df

    # print error messages so I can try and fix them and import them
    # idividually
    except Exception as e:
        print(drill)
        print(e)
        if 'utf-8\' codec can\'t decode byte' in str(e):
            print('HELLO')
            print('Reading French characters and creating')

            # grab the csv file I create with exell2csv.py
            excell2csved = glob.glob(calcihead +
                                     drill.split('/')[-1] +
                                     '/*.csv')

            print(excell2csved)
            df1 = pd.read_csv(excell2csved[0], sep=',')
            df2 = pd.read_csv(csvfile, sep=',', encoding='cp1252')

            del df1, df2

/work/armitagj/data/csvs/REP4/PEP2001
'NoneType' object is not subscriptable
/work/armitagj/data/csvs/REP4/PEP2002
'NoneType' object is not subscriptable
/work/armitagj/data/csvs/REP4/SDZ1289
'NoneType' object is not subscriptable
/work/armitagj/data/csvs/REP4/SUG1101
'NoneType' object is not subscriptable
SUG1302 : mesu.csv has 0 rows


In [72]:
file = '/work/armitagj/data/Excell_and_Photos/SDZ1255/GTR2010-04_0015_SDZ1255_Calcimétrie.xls'
df = pd.read_excel(file)
sf = df.iloc[:, 0]
sf2 = df.iloc[:, 1]
if 'Cote' in list(sf) or 'Cote (m)' in list(sf):

    if len(df.columns) < 5:
        print(file)

    if len(df.columns) > 7:
        for i in range(7, len(df.columns)):
            colname = f'Unnamed: {i}'
            df = df.drop([colname], axis=1)

    dfe = df.dropna().copy()
    dfe.columns = ['Cote', 'à 1 minute', 'à 4 minutes', 'à 19 minutes',
                   '% CaCO3', '% Dolomie', '% insolubles']

elif 'Cote toit' in list(sf) and 'Cote mur' in list(sf2):
    
    indx = list(sf).index('Cote toit')
    if np.isnan(list(sf)[indx + 2]):
        print('here')
        df = df.drop(df.columns[0], axis=1)
    else:
        print('here')
        df = df.drop(df.columns[1], axis=1)

    if len(df.columns) < 5:
        print(file)

    if len(df.columns) > 7:
        for i in range(7, len(df.columns)):
            colname = f'Unnamed: {i}'
            df = df.drop([colname], axis=1)


    dfe = df.dropna().copy()
    dfe.columns = ['Cote', 'à 1 minute', 'à 3 minutes', 'à 15 minutes',
                   '% CaCO3', '% Dolomie', '% insolubles']
    
elif 'Cote toit' in list(sf) and 'Cote mur' not in list(sf2):
    
    if len(df.columns) < 5:
        print(file)

    if len(df.columns) > 7:
        for i in range(7, len(df.columns)):
            colname = f'Unnamed: {i}'
            df = df.drop([colname], axis=1)


    dfe = df.dropna().copy()
    dfe.columns = ['Cote', 'à 1 minute', 'à 3 minutes', 'à 15 minutes',
                   '% CaCO3', '% Dolomie', '% insolubles']

else:
    dfe = pd.DataFrame(columns=['Cote', 'à 1 minute', 'à 3 minutes',
                                'à 15 minutes', '% CaCO3', '% Dolomie', '% insolubles'])

In [73]:
dfe

,Cote,à 1 minute,à 4 minutes,à 19 minutes,% CaCO3,% Dolomie,% insolubles
8,0.31,15,16,17,15,2,83
9,0.43,14,15,16,14,2,84
10,0.52,15,17,19,15,4,81
11,0.61,14,16,17,14,3,83
12,0.86,15,16,17,15,2,83
13,1.19,13,14,17,13,4,83


In [32]:
indx = list(sf).index('Cote toit')

In [33]:
indx

7

In [36]:
np.isnan(list(sf)[indx + 2])

False

In [81]:
csvfile = '/work/armitagj/data/csvs/REP4/SDZ1255/photos/imgs2.csv'
df2 = pd.read_csv(csvfile, sep=';')
len(df2.columns)

13